# Day 3: Knowledge-Based Robotic Decision-Making
Welcome to the third day of our hands-on course!
Today, you will focus on understanding how a knowledge base supports robotic decision-making. You’ll learn to query the knowledge base to identify necessary actions for tasks, like how to perceive the milk inside the fridge.
### Goal
By the end of the session, you will have successfully made queries to the knowledge base, enabling the robot to determine the steps required to complete its tasks.


## Theoretical Background
ToDo: We will provide an overview of how knowledge bases operate in the context of robotics. You’ll learn how the robot can understand the need to open the fridge door to perceive the milk inside.


## Step-by-Step Hands-On Exercises
### Query the Knowledge Base
Start by querying the knowledge base to understand how we can query the knowledge base.


In [ ]:
# Example code to query the knowledge base
# This is a placeholder example - replace with the actual code as needed
query = 'current_state(fridge, open).'
# Run the query on the knowledge base
result = knowledge_base.query(query)
print(result)

### Plan Execution

Import the necessary modules and define the objects in the environment, as we did in Day 2.

In [ ]:
from pycram.ros.tf_broadcaster import TFBroadcaster
from pycram.ros.viz_marker_publisher import VizMarkerPublisher, AxisMarkerPublisher
from pycram.worlds.bullet_world import BulletWorld
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.designators.object_designator import *
from pycram.datastructures.enums import ObjectType, WorldMode, TorsoState
from pycram.datastructures.pose import Pose
from pycram.process_module import simulated_robot, with_simulated_robot
from pycram.object_descriptors.urdf import ObjectDescription
from pycram.world_concepts.world_object import Object
from pycram.datastructures.dataclasses import Color
from pycram.designators.object_designator import BelieveObject

In [ ]:
extension = ObjectDescription.get_file_extension()
world = BulletWorld(WorldMode.DIRECT)
viz = VizMarkerPublisher()
tf = TFBroadcaster()

pr2 = Object("pr2", ObjectType.ROBOT, "pr2.urdf")
apartment = Object('apartment', ObjectType.ENVIRONMENT, f'apartment{extension}')
# milk = Object("milk", ObjectType.MILK, "milk.stl", pose=Pose([0.7,2.5, 0.9]))
#milk.color = Color(0, 0, 1, 1)
milk_desig = BelieveObject(names=["milk"])
robot_desig = BelieveObject(names=["pr2"])
apartment_desig = BelieveObject(names=["apartment"])

## Opening Action
Opening allows the robot to open a Container, the container is identified by an ObjectPart designator which describes the handle of the drawer that should be grasped. The OpeningAction needs to know which arm should be used to open the container. The ObjectPart would look like this:  

```python
ObjectPart(names=["frdige_door_link_handle"], part_of=apartment_desig)
```  

It takes the name of the handle as a string and the part_of designator of the apartment. This name is corresponding to the name of the handle in the URDF file. As you can tell naming all the parts in the URDF file is crucial for the robot to be able to interact with them, but knowing them during coding is quite annoying and frustrating. But we can use our knowledge base to ask for specific parts or names of objects.  


In [ ]:
# Placeholder code for querying the knowledge base for the name of the fridge door
query = 'get_name_handle(frdige..).'
actions = knowledge_base.query(query)
for action in actions:
    print(action)

Another side note, how does the robot now actual knows that it has to open fridge, obviously we have to tell it. But how? We can use the knowledge base to ask fo required actions. 

In [ ]:
# Placeholder code for querying the knowledge base for the name of the fridge door
query = 'required_action(robot, perceive, milk).'
actions = knowledge_base.query(query)
for action in actions:
    print(action)

now having the action we can use the OpeningAction to open the fridge door. so move the robot to the fridge door and open it!


In [ ]:
with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    NavigateAction([Pose([0, 1, 0], [0, 0, 0, 1])]).resolve().perform(
    handle_desig = ObjectPart(names=["handle_cab3_door_top"], part_of=apartment_desig.resolve())
    drawer_open_loc = AccessingLocation(handle_desig=handle_desig.resolve(),
                                        robot_desig=robot_desig.resolve()).resolve()
    NavigateAction([drawer_open_loc.pose]).resolve().perform()
    OpenAction(object_designator_description=handle_desig, arms=[Arms.RIGHT]).resolve().perform()
   

<details>

<summary>Click here to get the solution</summary>

```python

with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    NavigateAction([Pose([0, 1, 0], [0, 0, 0, 1])]).resolve().perform(
    handle_desig = ObjectPart(names=["handle_cab3_door_top"], part_of=apartment_desig.resolve())
    drawer_open_loc = AccessingLocation(handle_desig=handle_desig.resolve(),
                                        robot_desig=robot_desig.resolve()).resolve()
    NavigateAction([drawer_open_loc.pose]).resolve().perform()
    OpenAction(object_designator_description=handle_desig, arms=[Arms.RIGHT]).resolve().perform()
```
</details>

So what is missing look detect the milk and grasp it.


In [ ]:
with simulated_robot:
    

  
<details>

<summary>Click here to get the solution</summary>

```python

with simulated_robot:
      LookAtAction(targets=[milk_desig.resolve().pose]).resolve().perform()
    obj_desig = DetectAction(milk_desig).resolve().perform()
     ParkArmsAction([Arms.BOTH]).resolve().perform()
     PickUpAction(spoon_desig, [Arms.LEFT], [Grasp.TOP]).resolve().perform()

     ParkArmsAction([Arms.BOTH]).resolve().perform()
     
```
</details>